In [5]:
import h5py
import tensorflow as tf
from numpy import f2py

## Wrapping Fortran functions to Python

[Three ways to wrap](https://numpy.org/doc/stable/f2py/f2py.getting-started.html)

- Quick way: `python -m numpy.f2py -c eos.f90 -m eos`
- Smart way: `python -m numpy.f2py eos.f90 -m eos -h eos.pyf` to create the signature file `eos.pyf`, then `python -m numpy.f2py -c eos.pyf eos.f` to build the extension module `eos.


## Testing speed of tensor access

It takes too long to access a tensor's entries one-by-one; I need to avoid this when I'm passing fields into the EOS or optical depth routines

In [12]:
import time

n = 10
field = tf.zeros((n,n))
total = 0

start = time.time()

for i in range(n):
    for j in range(n):
        total += field[i,j]

duration = time.time() - start
rate = duration / n**2
print("Duration:", duration)
print("Time needed to access 1024^3 entries:", rate * 1024**3)

Duration: 0.025780200958251953
Time to access 1024^3 entries: 276812.8


## Testing tf gradients

In [13]:
tf.math.pow(z, 2)

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [11]:
z = tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(z) # without this line, grad is None
    a = tf.divide(1, z+1)
    print('a:', a)
    
grad = tape.gradient(a, z)
print('grad:', grad)

a: tf.Tensor(0.25, shape=(), dtype=float32)
grad: tf.Tensor(-0.0625, shape=(), dtype=float32)


# Load in the data

In [1]:
filename = "../../../../../cscratch1/sd/jupiter/sim2_z3_FGPA_cgs.h5"

#snap = h5py.File(filename,'r')

## Testing classes

In [2]:
import snapshot

snap = snapshot.Snapshot(filename)

In [3]:
# test snapshot methods
temp = snap.read_field('/native_fields/temperature')
type(temp)

tensorflow.python.ops.resource_variable_ops.ResourceVariable

In [12]:
# test the universe class (from universe.py)
u = snap.univ
chi = 10
z = 1
u.chi_to_proper_cgs(chi, z)

<tf.Tensor: shape=(), dtype=float64, numpy=2.2856888888888885e+25>

## Metadata

In [6]:
shape = snap['domain'].attrs['shape']
size = snap['domain'].attrs['size']

z = snap['universe'].attrs['redshift']
omega_b = snap['universe'].attrs['omega_b']
omega_m = snap['universe'].attrs['omega_m']
omega_l = snap['universe'].attrs['omega_l']
h = snap['universe'].attrs['hubble']

scale_factor = 1.0 / (1.0 + z)

In [ ]:
# TODO: (possibly) implement a grid class
# grid = new Grid(shape[0], shape[1], shape[2], size[0], size[1], size[2])

In [7]:
#snap.close()